# Desafío de Ingeniería de Datos — Análisis de Tweets

## 1. Preparación del entorno

En esta sección, aseguramos que tenemos los archivos necesarios, descargamos los datos desde drive si es necesario, convertimos en parquet para mejor aproveitamento con DuckDB y respondemos las preguntas.

# Contexto y enfoque para resolver el desafío

Este notebook documenta el proceso completo para resolver tres preguntas relacionadas con el análisis de un conjunto de tweets. A continuación se explica la lógica, las decisiones técnicas y las bibliotecas utilizadas para construir una solución eficiente y escalable.

## Elección de herramientas y bibliotecas

- **DuckDB:** Elegí DuckDB como motor de consultas SQL embebido en Python por varias razones:
  - Permite ejecutar consultas SQL directamente sobre archivos Parquet y JSON sin necesidad de cargas complejas.
  - Es muy eficiente para análisis de datos en columnas (columnar), lo que mejora el rendimiento al trabajar con grandes volúmenes de datos.
  - Proporciona un lenguaje declarativo (SQL) para consultas complejas, facilitando la extracción y agregación de información.
  - Comparado con leer archivos directamente con pandas, DuckDB es más rápido y consume menos memoria cuando se manejan grandes datasets.
  - Evita la necesidad de construir procesos ETL o pipelines externos, manteniendo todo en un solo entorno de código.

- **gdown:** Para descargar archivos desde Google Drive de forma automatizada y reproducible.

- **pandas:** Usado solo para conversión rápida de JSON a Parquet, dado que DuckDB no soporta JSON directamente para consultas.

## Razonamiento general del enfoque

1. **Preparación de datos:** 
   - El dataset original está en formato JSON, con tweets almacenados línea a línea.
   - Para consultas eficientes, convertimos este JSON a Parquet, un formato columnar optimizado para consultas analíticas.

2. **Consultas analíticas:**
   - Utilizo SQL para filtrar, agrupar y ordenar los datos, logrando respuestas rápidas y claras a las preguntas.
   - Uso expresiones regulares para extraer emojis y menciones dentro del texto de los tweets.
   - Extraigo solo los datos relevantes para cada pregunta, mejorando el rendimiento.

3. **Ventajas de esta solución:**
   - Código limpio y mantenible.
   - Uso de tecnologías que permiten escalabilidad a datasets más grandes.
   - Facilidad para modificar y extender consultas futuras.

---


In [1]:
from download_data import download_file_from_google_drive

In [2]:
download_file_from_google_drive()

✅ File already exists. Skipping download.
